In [1]:
from math import cos, sin, tan, acos, asin, atan, atan2, degrees, radians
from numpy import reciprocal, pi, sqrt, angle, exp
import numpy as np
import time, sys

#hextime=float.hex(time.time())

# The foregoing is a clunky way to do this
# Haven't spent a lot of time thinking through a more elegant solution

def cosd(x):
  return cos(radians(x))

def sind(x):
  return sin(radians(x))

def tand(x):
  return tan(radians(x))

def acosd(x):
  return degrees(acos(x))

def asind(x):
  return degrees(asin(x))

def atand(x):
  return degrees(atan(x))

def atan2d(y, x):
  return degrees(atan2(y, x))

## Now define conversion functions:

def P2R(radii, angles):
    return radii * exp(1j*angles*(pi/180))

def R2P(x):
    return abs(x), angle(x, deg=True)


print("Author        : S Monroe")
print("Version       :",sys.version[0:66])
print("NumPy lib     :",np.version.version)
print("Encoding      :",sys.getdefaultencoding())
print("Timestamp     :",time.time())

Author        : S Monroe
Version       : 3.7.3 | packaged by conda-forge | (default, Mar 27 2019, 23:18:50)
NumPy lib     : 1.16.4
Encoding      : utf-8
Timestamp     : 1571224747.0851128


# Junction FET Biasing

All JFET devices follow a "square law" relationship:<br><br>
$$I_D=I_{DSS} \cdot \left ( 1-\frac{V_{GS}}{{V_P}} \right )^2$$

Because of this characteristic, the JFET lends itself well to solutions with the quadratic formula,

$$\begin{array}{*{20}c} {x = \frac{{ - b \pm \sqrt {b^2 - 4ac} }}{{2a}}} & {{\rm{when}}} & {ax^2 + bx + c = 0} \\ \end{array}$$

<img src="Parabola_Anatomy.jpg" alt="Parabola Anatomy " width="400" height="400" title="Parabola title">

It is possible to solve for drain current, $I_D$ by substituting $I_D$ for $x$ ...<br>

Depending upon the biasing scheme involved, it is possible to substitute circuit parameters for $a$, $b$, and $c$ ...

# Self Bias

Use the schematic shown below,

![JFET_Self_Bias](self_bias.JPG "Junction FET Biased using self-bias")

The formula $$ax^2+bx+c=0$$ contains three terms,<br><br>
$a$: quadratic coefficient<br>
$b$: linear coefficient<br>
$c$: constant or free term<br>

Since, 
$$V_{GS}=I_D \cdot R_S$$<br>
and,
$$I_D=I_{DSS} \cdot \left ( 1-\frac{V_{GS}}{{V_P}} \right )^2$$<br>
substitute
$$I_D=I_{DSS} \cdot \left ( 1-\frac{I_D \cdot R_S}{{V_P}} \right )^2$$
Solve for $I_D$ ...
$$I_D={\frac {-b - {\sqrt {b^{2}-4ac\ }}}{2a}}$$

When a JFET is biased using self-bias, as shown above, $$a=R_S^2$$ ... i.e., the square of the resistor value connected to the source.<br><br> $$b=-\left ( 2 \cdot \left | V_P \right | \cdot R_S +  \left ( \frac{V_P^2}{I_{DSS}} \right ) \right)$$<br><br>
$$c=V_P^2$$

In [2]:
VDD=15.
RS=1000.
RD=4.7E3
RG=0.1
IDSS=17.5E-3
VP=-1.75
a=RS**2
b=-(2*abs(VP)*RS+(VP**2/IDSS))
c=VP**2
print("a=",a)
print("b=",b)
print("c=",c)
#
#
#uncomment lines for testing correspondence 
coeff = [a, b, c]
ans=np.roots(coeff)
print(ans)

a= 1000000.0
b= -3675.0
c= 3.0625
[0.00239777 0.00127723]


In [3]:
disc=(b**2-4*a*c)
dnom=2*a
ID=(-b-sqrt(disc))/(2*a)

In [4]:
VDS=VDD-ID*(RS+RD)
VS=ID*RS
VG=0.  #self-bias only!
VD=VDD-ID*RD

In [5]:
print(" ID:",'{0:.4e}'.format(ID),"Amperes")
print(" VS:",'{0:.4f}'.format(VS),"Volts")
print(" VG:",'{0:.4f}'.format(VG),"Volts")
print(" VD:",'{0:.4f}'.format(VD),"Volts")
print("VDS:",'{0:.4f}'.format(VDS),"Volts")

 ID: 1.2772e-03 Amperes
 VS: 1.2772 Volts
 VG: 0.0000 Volts
 VD: 8.9970 Volts
VDS: 7.7198 Volts


# Voltage Divider Bias

Use the schematic shown below,

![JFET_Voltage_Divider_Bias](JFET_VDB.JPG "Junction FET Biased using voltage divider bias")

In [6]:
VDD=15.
R1=220.0E3
R2=10.0E3
RD=4700.0
RS=1500.0
IDSS=20.21E-3
VP=-1.44

$$V_G=V_{DD} \cdot \frac{R_2}{R_1+R_2}$$<br><br>
$$V_{GS}=V_{DD} \cdot \frac{R_2}{R_1+R_2}-I_DR_S$$<br><br>
$$I_D={\frac {-b - {\sqrt {b^{2}-4ac\ }}}{2a}}$$
where,<br><br> $$a=R_S^2$$<br><br> $$b=-\left [ 2\left ( \left | V_P \right | +\left | V_G \right | \right )R_S+\left ( \frac{V_P^2} {I_{DSS}} \right ) \right ]$$<br><br>
$$c=\left ( \left | V_P \right |+\left | V_G \right | \right )^2$$

In [12]:
VDD=15.0
RS=1500.0
RD=4700.0
R1=220.0E3
R2=10.0E3
IDSS=20.21E-3
VP=-1.44
VG=VDD*(R2/(R1+R2))
print(VG)
a=RS**2
b=-(2*(abs(VP)+abs(VG))*RS+(VP**2/IDSS))
c=(abs(VP)+abs(VG))**2
print("a=",a)
print("b=",b)
print("c=",c)
print("VG=",VG)
#
#
# uncomment lines for testing correspondence 
coeff = [a, b, c]
ans=np.roots(coeff)
print(ans)

0.6521739130434783
a= 2250000.0
b= -6379.124411075016
c= 4.377191682419658
VG= 0.6521739130434783
[0.00167081 0.00116436]


In [13]:
disc=(b**2-4*a*c)
dnom=2*a
ID=(-b-sqrt(disc))/(2*a)

In [14]:
VGS=(VDD*(R2/(R1+R2)))-ID*RS
VD=VDD-(ID*RD)
VS=ID*RS
VDS=VDD-ID*(RD+RS)

In [15]:
print(" ID:",'{0:.4e}'.format(ID),"Amperes")
print(" VS:",'{0:.4f}'.format(VS),"Volts")
print(" VG:",'{0:.4f}'.format(VG),"Volts")
print(" VD:",'{0:.4f}'.format(VD),"Volts")
print("VDS:",'{0:.4f}'.format(VDS),"Volts")
print("VGS:",'{0:.4f}'.format(VGS),"Volts")

 ID: 1.1644e-03 Amperes
 VS: 1.7465 Volts
 VG: 0.6522 Volts
 VD: 9.5275 Volts
VDS: 7.7810 Volts
VGS: -1.0944 Volts


When $V_{GS}=0$,<br>

In [16]:
ID=VG/RS
print(" ID:",'{0:.4e}'.format(ID),"Amperes when VGS=0")

 ID: 4.3478e-04 Amperes when VGS=0


In [7]:
import numpy as np
VDD=15.
RS=1000.0
RD=4.7E3
RG=0.0
IDSS=14.2E-3
VP=-1.442
a=RS**2
b=-(2*abs(VP)*RS+(VP**2/IDSS))
c=VP**2
print("a=",a)
print("b=",b)
print("c=",c)
#
#
#uncomment lines for testing correspondence 
coeff = [a, b, c]
ans=np.roots(coeff)
print(ans)

a= 1000000.0
b= -3030.4340845070424
c= 2.079364
[0.00198053 0.0010499 ]
